# Notebook 14: Agentic Workflows - MCP Basics

**Learning Objectives:**
- Understand Model Context Protocol (MCP)
- Set up MCP servers and clients
- Connect local LLMs to external tools
- Build your first tool-using agent

## Prerequisites

### Hardware Requirements

| Model Option | Model Name | Size | Min RAM | Recommended Setup | Notes |
|--------------|------------|------|---------|-------------------|-------|
| **small (CPU-friendly)** | llama3.2:1b | 1.3GB | 8GB | 8GB RAM, CPU | Fast, good for learning |
| **large (GPU-optimized)** | llama3.1:8b | 4.7GB | 16GB | 12GB VRAM (RTX 4080) | Better reasoning |
| **SOTA (reference only)** | Claude 3.5 Sonnet | API | N/A | API key required | Production-grade |

### Software Requirements
- Python 3.10+
- Ollama installed (see Notebook 10)
- Libraries: `mcp`, `ollama`, `pydantic`

### Installation

```bash
# Install MCP SDK
pip install mcp

# Install Ollama Python library
pip install ollama

# Pull Ollama models
ollama pull llama3.2:1b
ollama pull llama3.1:8b  # Optional, if you have resources
```

## What is MCP?

**Model Context Protocol (MCP)** is an open protocol created by Anthropic for connecting AI assistants to external tools and data sources.

### Why MCP?

**Traditional Problem:**
- Each AI application needs custom integrations for every tool
- No standard way to expose tools to LLMs
- Tight coupling between LLM and tool code

**MCP Solution:**
- **Standardized protocol** for tool communication
- **Decoupled architecture** - servers expose tools, clients consume them
- **Reusable servers** - Write once, use with any MCP client
- **Language agnostic** - Servers in Python, clients in any language

### MCP Architecture

```
┌─────────────────┐
│   AI Assistant  │  (Your LLM - Ollama, Claude, etc.)
│   (MCP Client)  │
└────────┬────────┘
         │ MCP Protocol (JSON-RPC)
         │
┌────────▼────────┐
│   MCP Server    │  (Exposes tools/resources)
│  - Weather API  │
│  - Calculator   │
│  - File System  │
└─────────────────┘
```

### Key Concepts

1. **MCP Server** - Exposes tools, resources, and prompts
2. **MCP Client** - Connects to servers, invokes tools
3. **Tools** - Functions the LLM can call (e.g., `search`, `calculate`)
4. **Resources** - Data sources (files, databases, APIs)
5. **Prompts** - Reusable prompt templates

## Expected Behaviors

### First Time Running
- **Ollama Model Pull**: ~1.3GB for llama3.2:1b (~2-5 minutes)
- **MCP Installation**: ~10MB for mcp SDK
- Models cached in `~/.ollama/models/`

### MCP Server Startup
```
Starting MCP server...
Server running on stdio
Registered tools: ['calculator', 'get_weather']
```

### Tool Invocation
- LLM receives tool descriptions
- LLM decides when to use tools
- Tool results returned to LLM
- LLM generates final response

### Performance
- **llama3.2:1b** (CPU): 2-5 seconds per turn
- **llama3.1:8b** (GPU): 1-3 seconds per turn
- Tool execution: 100-500ms depending on tool

### Common Observations
- Smaller models may not always use tools correctly
- Clear tool descriptions improve usage
- Multi-step reasoning works better with larger models
- Tool results should be concise for better understanding

In [ ]:
import json
import random
import ollama
from typing import Any
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
random.seed(1103)

print("MCP Tutorial - Setup Complete")
print(f"Ollama available: {True}")

## Model Selection

In [ ]:
# CHOOSE YOUR MODEL:

# Option 1: small model (CPU-friendly, fast)
MODEL_NAME = "llama3.2:1b"  # 1.3GB, good for learning

# Option 2: large model (GPU-optimized, better reasoning)
# MODEL_NAME = "llama3.1:8b"  # 4.7GB, more capable

print(f"Selected model: {MODEL_NAME}")

## Building Your First MCP Tool

Let's create a simple calculator tool that the LLM can use.

In [ ]:
# Define tools in the format Ollama expects
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'calculator',
            'description': 'Perform basic arithmetic operations (add, subtract, multiply, divide)',
            'parameters': {
                'type': 'object',
                'properties': {
                    'operation': {
                        'type': 'string',
                        'description': 'The operation to perform',
                        'enum': ['add', 'subtract', 'multiply', 'divide']
                    },
                    'a': {
                        'type': 'number',
                        'description': 'First number'
                    },
                    'b': {
                        'type': 'number',
                        'description': 'Second number'
                    }
                },
                'required': ['operation', 'a', 'b']
            }
        }
    }
]

print("Tools defined:")
print(f"  - {tools[0]['function']['name']}: {tools[0]['function']['description']}")

In [ ]:
# Implement the calculator function
def calculator(operation: str, a: float, b: float) -> float:
    """Execute calculator operations."""
    if operation == 'add':
        return a + b
    elif operation == 'subtract':
        return a - b
    elif operation == 'multiply':
        return a * b
    elif operation == 'divide':
        if b == 0:
            return "Error: Division by zero"
        return a / b
    else:
        return "Error: Unknown operation"

# Map function names to implementations
available_functions = {
    'calculator': calculator
}

print("Function implementations ready")

## Agent Loop with Tool Calling

This is the core agent loop that allows the LLM to use tools.

In [ ]:
def run_agent(prompt: str, model: str = MODEL_NAME, max_iterations: int = 5) -> str:
    """
    Run an agent that can use tools to answer questions.
    
    Args:
        prompt: User question
        model: Ollama model to use
        max_iterations: Maximum tool-calling iterations
    
    Returns:
        Final answer from the agent
    """
    messages = [{'role': 'user', 'content': prompt}]
    
    print(f"\n{'='*70}")
    print(f"User: {prompt}")
    print(f"{'='*70}\n")
    
    for iteration in range(max_iterations):
        # Call LLM with tools
        response = ollama.chat(
            model=model,
            messages=messages,
            tools=tools
        )
        
        messages.append(response['message'])
        
        # Check if LLM wants to use a tool
        if not response['message'].get('tool_calls'):
            # No tool call, we have final answer
            final_answer = response['message']['content']
            print(f"Agent: {final_answer}")
            print(f"\n{'='*70}")
            return final_answer
        
        # Process tool calls
        for tool_call in response['message']['tool_calls']:
            function_name = tool_call['function']['name']
            function_args = tool_call['function']['arguments']
            
            print(f"Tool Call: {function_name}({function_args})")
            
            # Execute the function
            function_to_call = available_functions[function_name]
            function_response = function_to_call(**function_args)
            
            print(f"Tool Result: {function_response}\n")
            
            # Add tool result to messages
            messages.append({
                'role': 'tool',
                'content': str(function_response)
            })
    
    return "Maximum iterations reached"

print("Agent function ready")

## Example 1: Simple Calculation

In [ ]:
result = run_agent("What is 127 multiplied by 83?")

## Example 2: Multi-Step Calculation

In [ ]:
result = run_agent("If I have $100 and buy 3 items at $15 each, how much money do I have left?")

## Example 3: Multiple Operations

In [ ]:
result = run_agent("Calculate (25 + 17) * 3")

## Adding More Tools

Let's add a weather tool to demonstrate multiple tools.

In [ ]:
# Add weather tool
tools.append({
    'type': 'function',
    'function': {
        'name': 'get_weather',
        'description': 'Get current weather for a city',
        'parameters': {
            'type': 'object',
            'properties': {
                'city': {
                    'type': 'string',
                    'description': 'City name'
                }
            },
            'required': ['city']
        }
    }
})

# Implement weather function (mock data for demo)
def get_weather(city: str) -> str:
    """Get mock weather data."""
    weather_data = {
        'san francisco': 'Sunny, 72°F',
        'new york': 'Cloudy, 65°F',
        'london': 'Rainy, 58°F',
        'tokyo': 'Clear, 75°F'
    }
    city_lower = city.lower()
    return weather_data.get(city_lower, f"Weather data not available for {city}")

# Update available functions
available_functions['get_weather'] = get_weather

print("Weather tool added")
print(f"Total tools: {len(tools)}")

In [ ]:
result = run_agent("What's the weather in San Francisco?")

## Comparison: With vs Without Tools

In [ ]:
# Without tools - LLM guesses
print("WITHOUT TOOLS:")
response_no_tools = ollama.chat(
    model=MODEL_NAME,
    messages=[{'role': 'user', 'content': 'What is 9876 multiplied by 5432?'}]
)
print(f"Answer: {response_no_tools['message']['content']}")
print(f"\nActual: {9876 * 5432}")

print("\n" + "="*70 + "\n")

# With tools - LLM uses calculator
print("WITH TOOLS:")
result = run_agent("What is 9876 multiplied by 5432?")

## Understanding Tool Selection

The LLM decides which tool to use based on descriptions.

In [ ]:
# Test tool selection with different queries
test_queries = [
    "What's 42 divided by 7?",
    "Is it raining in London?",
    "Calculate the sum of 123 and 456, then tell me the weather in Tokyo"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    result = run_agent(query)
    print()

## Exercises

1. **New Tool**: Add a `get_time` tool that returns the current time for a timezone
2. **Error Handling**: Test division by zero and observe how the agent handles it
3. **Complex Query**: Ask a question that requires multiple tool calls
4. **Tool Description**: Modify tool descriptions and see how it affects tool selection
5. **Model Comparison**: Compare llama3.2:1b vs llama3.1:8b tool usage accuracy

In [ ]:
# Your code here for exercises


## Key Takeaways

✅ **MCP** provides a standard protocol for AI-tool integration

✅ **Tools** are defined with name, description, and parameters

✅ **LLMs decide** when to use tools based on descriptions

✅ **Agent loop** handles iterative tool calling

✅ **Local models** (Ollama) work well for basic tool use

## Next Steps

- Try **Notebook 15**: Building MCP Servers with local LLMs
- Explore [MCP documentation](https://modelcontextprotocol.io/)
- Learn about [Ollama function calling](https://ollama.com/blog/tool-support)

## Resources

- [Model Context Protocol](https://modelcontextprotocol.io/)
- [MCP Specification](https://spec.modelcontextprotocol.io/)
- [Ollama Tool Support](https://ollama.com/blog/tool-support)
- [MCP GitHub](https://github.com/modelcontextprotocol)